# 承接rag-pre的进阶操作

**包括链和agent，实现历史对话**

In [3]:
import getpass
import os

os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT_ID"] = "project-1"
os.environ["LANGSMITH_PROJECT_NAME"] = "example rag"
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = "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"

from langchain_deepseek.chat_models import ChatDeepSeek  # 导入 DeepSeek 的聊天模型
from langchain_huggingface import HuggingFaceEmbeddings # 导入 HuggingFace 的嵌入模型
from langchain.chains import RetrievalQA # 导入检索问答链
from langchain.prompts import PromptTemplate # 导入提示模板
from langchain.chains import LLMChain # 导入 LLM 链
from langchain.memory import ConversationBufferMemory # 导入对话缓冲区内存
from langchain_chroma import Chroma # 导入 Chroma 向量存储


deepseek_api_key = "sk-fffbb9b8a78d436a91a4780356b67a93"

llm = ChatDeepSeek(model="deepseek-chat", api_key = deepseek_api_key)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)